#### Usage to plot similarity matrix

In [ ]:
#for relative open masslib
import sys
sys.path.append('..')
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from nhsmasslib.mass import MassSpectrum
from nhsmasslib.mass import Tmds
from nhsmasslib.mass import MassSpectrumList
import nhsmasslib.draw as draw

#### Load spectra

In [ ]:
masses = []
names = []
for filename in sorted(os.listdir("data/similarity/")):
    if filename[-3:] != 'csv':
        continue
    names.append(filename[:-4])
    masses.append(MassSpectrum().load(f"data/similarity/{filename}", assign_mark=True, metadata={'name':filename}))

#### Create MassSpectrumList instance

In [ ]:
collection = MassSpectrumList(masses)

#### Calc and Plot similarity matrix

In [ ]:
collection.calculate_similarity()


In [ ]:
collection.draw_similarity()

#### Batch processing


In [ ]:
squares = pd.DataFrame()

for i, spec in enumerate(collection):

    spec = spec.recallibrate(how='mdm')
    spec = spec.assign(sign='0')
    try:
        spec = spec.assign_by_tmds(max_num=100)
    except:
        pass

    draw.spectrum(spec)
    plt.title(collection.names[i])
    
    collection.spectra[i] = spec

    draw.vk(spec)
    plt.title(f'{collection.names[i]}, {len(spec.drop_unassigned())} formulas')

    square = spec.vk_squares()
    #squares[collection.names[i]] = square['value']

squares.index = [i for i in range(1,21)]
print(squares)

#### We can also calculate TMDS-spectrum and look simmmilarity of its

In [ ]:
tmd = MassSpectrumList([], collection.names)

for i, spec in enumerate(collection.spectra):
    tmds = Tmds().calc(spec, p=0.2, C13_filter = True) #by varifiy p-value we can choose how much mass-diff we will take
    tmds = tmds.assign(brutto_dict={'C':(-1,20),'H':(-4,40), 'O':(-1,20),'N':(0,1)})
    tmds = tmds.calculate_mass()
    tmd.spectra.append(tmds)

tmd.draw_similarity()  